In [5]:
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql.functions._
import org.apache.spark.rdd.RDD
import scala.util.parsing.json.JSON
import org.apache.spark.sql.DataFrame

lastException = null


null

In [6]:
// Load movies.csv from GCP Cloud Storage as a DataFrame
val moviePath = "gs://priyanshi-spark-bucket-3/movie.csv"
val moviesDF = spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv(moviePath)
    .cache()

moviePath = gs://priyanshi-spark-bucket-3/movie.csv
moviesDF = [movieId: int, title: string ... 1 more field]


[movieId: int, title: string ... 1 more field]

In [7]:
import org.apache.spark.sql.functions._
val duplicateMoviesDF = moviesDF
  .groupBy(moviesDF.columns.map(col): _*) // Group by all columns
  .count()
  .filter($"count" > 1) // Keep only duplicates
duplicateMoviesDF.show()

+-------+-----+------+-----+
|movieId|title|genres|count|
+-------+-----+------+-----+
+-------+-----+------+-----+



duplicateMoviesDF = [movieId: int, title: string ... 2 more fields]


[movieId: int, title: string ... 2 more fields]

In [8]:
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._
// Define a schema for the movies
val schema = moviesDF.schema
// Create duplicate rows manually
val duplicateRows = Seq(
  Row(1, "Toy Story (1995)", "Adventure|Animation|Children|Comedy"),
  Row(2, "Jumanji (1995)", "Adventure|Children|Fantasy"),
    Row(7, "Sabrina (1995)", "Comedy|Romance"),
    Row(13, "Nixon (1995)", "Drama")
)
// Convert duplicate rows to a DataFrame
val duplicateDF = spark.createDataFrame(
  spark.sparkContext.parallelize(duplicateRows),
  schema
)
// Union the original DataFrame with the duplicate DataFrame
val moviesWithDuplicatesDF = moviesDF.union(duplicateDF)
moviesWithDuplicatesDF.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

schema = StructType(StructField(movieId,IntegerType,true),StructField(title,StringType,true),StructField(genres,StringType,true))
duplicateRows = List([1,Toy Story (1995),Adventure|Animation|Children|Comedy], [2,Jumanji (1995),Adventure|Children|Fantasy], [7,Sabrina (1995),Comedy|Romance], [13,Nixon (1995),Drama])
duplicateDF = [movieId: int, title: string ... 1 more field]
moviesWithDuplicatesDF = [movieId: int, title: string ... 1 more field]


[movieId: int, title: string ... 1 more field]

In [9]:
import org.apache.spark.sql.functions._
val duplicatesDF = moviesWithDuplicatesDF
  .groupBy("movieId", "title") // Group by movieId and title
  .count()                     // Count occurrences
  .filter($"count" > 1)        // Filter rows where count > 1
duplicatesDF.show()

+-------+----------------+-----+
|movieId|           title|count|
+-------+----------------+-----+
|      2|  Jumanji (1995)|    2|
|      1|Toy Story (1995)|    2|
|      7|  Sabrina (1995)|    2|
+-------+----------------+-----+



duplicatesDF = [movieId: int, title: string ... 1 more field]


[movieId: int, title: string ... 1 more field]

In [10]:
val duplicateMoviesDF = moviesDF.union(duplicatesDF)
duplicateMoviesDF.write
  .option("header", "true")
  .mode("overwrite") // Overwrite the existing file
  .csv("hdfs:///user/priyanshi-spark-bucket-3/duplicated_movie.csv")

duplicateMoviesDF = [movieId: int, title: string ... 1 more field]


[movieId: int, title: string ... 1 more field]

In [11]:
val moviesPath = "hdfs:///user/priyanshi-spark-bucket-3/duplicated_movie.csv"
val moviesDF = spark.read.option("header", "true").csv(moviesPath)
//Convert to RDD with composite key (movieId, title) and Remove duplicate records
val cleanedMoviesDF = moviesDF.rdd.map(row => {
  val movieId = row.getString(row.fieldIndex("movieId"))
  val title = row.getString(row.fieldIndex("title"))
  val genres = row.getString(row.fieldIndex("genres"))
  ((movieId, title), genres) // Key: (movieId, title), Value: genres
}).reduceByKey((genres1, genres2) => s"$genres1|$genres2").map {
  case ((movieId, title), combinedGenres) => (movieId, title, combinedGenres)
}.toDF("movieId", "title", "genres")

moviesPath = hdfs:///user/priyanshi-spark-bucket-3/duplicated_movie.csv
moviesDF = [movieId: string, title: string ... 1 more field]
cleanedMoviesDF = [movieId: string, title: string ... 1 more field]


[movieId: string, title: string ... 1 more field]

In [12]:
// // Step 4: Validation
val originalCount = moviesDF.count()
val deduplicatedCount = cleanedMoviesDF.count()
val duplicatesRemoved = originalCount - deduplicatedCount
println(s"Original record count: $originalCount")
println(s"Deduplicated record count: $deduplicatedCount")
println(s"Duplicates removed: $duplicatesRemoved")

Original record count: 27281
Deduplicated record count: 27278
Duplicates removed: 3


originalCount = 27281
deduplicatedCount = 27278
duplicatesRemoved = 3


3

In [13]:
%AddDeps org.apache.spark spark-avro_2.12 3.3.2 --transitive

Marking org.apache.spark:spark-avro_2.12:3.3.2 for download
Obtained 12 files
Marking org.apache.spark:spark-avro_2.12:3.3.2 for download
Obtained 12 files


In [14]:
val outputPath = s"gs://priyanshi-spark-bucket-3/cleaned_movies.avro"
cleanedMoviesDF.write
  .format("avro")
  .mode("overwrite")
  .save(outputPath)
println("Cleaned movies data saved successfully in Avro format.")

Cleaned movies data saved successfully in Avro format.


outputPath = gs://priyanshi-spark-bucket-3/cleaned_movies.avro


gs://priyanshi-spark-bucket-3/cleaned_movies.avro